# Problem 6

In [1]:
import hickle as hkl
import math

# Loading Phi matrix m*100
Phi = hkl.load('./p2_Phi_34.hkl')  # m=10/34/80
m=Phi.shape[0]

# Load training examples
info = hkl.load('p2_data_1000_dim_100.hkl')
x_mat = info['noisy_x']   # noisy signal (100x1000)
xs_mat = info['sparse_x'] # sparse signal (100x1000)
z_mat = info['label']    # one-hot true encoded labels (3x1000)

# Projection of training examples to m dimensions
p = 1/math.sqrt(m) * Phi @ x_mat
x_mat[:,0]

array([-4.33330339e+00, -1.00013421e+00,  1.41451010e+00,  3.37927447e-05,
        9.99993226e-01, -9.09007312e-05,  9.99941049e-01, -3.33340110e+00,
       -1.41440029e+00,  9.99951504e-01, -9.99764599e-01, -4.95420813e-01,
        1.82890530e+00, -9.99822425e-01,  2.41445332e+00, -1.41444551e+00,
        3.17542531e-05, -1.02740514e-04, -1.41445668e+00,  4.14600670e-01,
       -1.41437867e+00,  1.50448669e+00, -1.96234976e-05,  4.05287151e-05,
        8.39231173e-05, -3.33341829e+00,  1.91894202e+00,  1.41448006e+00,
        2.41434835e+00,  1.41440749e+00,  2.82874310e+00,  1.41430598e+00,
       -1.41443017e+00,  1.41469546e+00,  1.70006279e-04, -1.41428164e+00,
        1.41439167e+00, -5.04499752e-01,  1.00007580e+00, -1.82892239e+00,
       -9.99987677e-01, -4.14354391e-01,  1.41422766e+00, -1.91876161e+00,
       -2.91891447e+00,  2.41443836e+00, -1.41446241e+00, -2.41436487e+00,
       -5.23301330e-05,  1.41451312e+00,  1.41442414e+00, -6.05081193e-05,
       -2.82894315e+00, -

# Problem 7

In [2]:
# Loading {U} vectors 100x6
u_mat = hkl.load('u_mat_rev.hkl')

# {U} projected to m dimensional spae
u_mat_proj = 1/(math.sqrt(m)) * Phi @ u_mat


# Comparing squared Euclidean distances between original and projected {U} vectors
import numpy as np

loss = 0
norm_loss = 0
total_pair_dist = 0
for i in range(6):
    for j in range(6):
        if i!=j:
            # Calculating squared euclidean distance (l2 norm ^2 ) for ith and jth vectors of u_mat
            d_dim_Edist = np.linalg.norm(u_mat[:,i]-u_mat[:,j])**2
        
            # Calculating squared euclidean distance (l2 norm ^2 ) for ith and jth vectors of u_mat_proj
            m_dim_Edist = np.linalg.norm(u_mat_proj[:,i]-u_mat_proj[:,j])**2
            print(i+1,j+1,d_dim_Edist, m_dim_Edist,abs(d_dim_Edist-m_dim_Edist))

            # loss
            #total_pair_dist += d_dim_Edist
            loss += abs(d_dim_Edist-m_dim_Edist)
            norm_loss += (abs(d_dim_Edist-m_dim_Edist))/d_dim_Edist
    print('\n')

print('% Loss: ',loss/30)
print('% Normalized Loss: ', norm_loss/30)

1 2 74.0 66.82352941176471 7.17647058823529
1 3 71.0 98.41176470588232 27.41176470588232
1 4 67.0 49.70588235294117 17.294117647058833
1 5 55.0 51.117647058823515 3.882352941176485
1 6 72.99999999999999 57.4705882352941 15.529411764705884


2 1 74.0 66.82352941176471 7.17647058823529
2 3 77.00000000000001 80.76470588235293 3.7647058823529136
2 4 77.00000000000001 80.29411764705881 3.2941176470587976
2 5 55.0 35.58823529411764 19.411764705882362
2 6 71.0 71.11764705882351 0.11764705882350768


3 1 71.0 98.41176470588232 27.41176470588232
3 2 77.00000000000001 80.76470588235293 3.7647058823529136
3 4 68.0 93.99999999999997 25.99999999999997
3 5 54.0 47.29411764705881 6.705882352941188
3 6 74.0 95.88235294117645 21.88235294117645


4 1 67.0 49.70588235294117 17.294117647058833
4 2 77.00000000000001 80.29411764705881 3.2941176470587976
4 3 68.0 93.99999999999997 25.99999999999997
4 5 56.0 56.588235294117645 0.588235294117645
4 6 64.0 56.235294117647044 7.764705882352956


5 1 55.0 51.11764

# Problem 8

In [3]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


# Function to generate one-hot encoding of the KMeans Clustering algorithm
def oh_encode(prediction, n_cluster):
    one_hot_result = np.zeros((len(prediction),n_cluster))
    for i in range(len(prediction)):
        one_hot_result[i,prediction[i]]=1
    return one_hot_result.T



# x_mat contains 1000 samples of d-dimensional data
# p contains projection of x_mat to m dimensions
num_clusters = 3 # varied 2/3/4/5
kmeans = KMeans(init='k-means++', n_clusters=num_clusters).fit(p.T)  # fit takes data as (num_samples x num_features)
m_k = kmeans.cluster_centers_ # cluster-centers
a_mat = oh_encode(kmeans.labels_,num_clusters)  # one-hot encoded labels (num_clusters x 1000)

In [10]:
# Finding emperical probabilities P[a_i[k]=1 | z_i[l]=1]  for 3xK table
# a_mat -> labels predicted by KMeans
# z_mat -> ground truth labels
cnt = np.zeros(num_clusters) # components in each predicted cluster for a given $num_clusters$ 
# [319 350 331] # component wise sample_pt count

comp_label = np.array([[1,0,0],
                      [0,1,0],
                      [0,0,1]])

total_comp_cnt = np.array([319,350,331])

for j in range(3):
    cnt = np.zeros(num_clusters)
    for i in range(len(z_mat.T)):
        if np.array_equal(np.array([1,0,0]),a_mat[:,i].reshape(num_clusters)) and np.array_equal(comp_label[j],z_mat[:,i].reshape(3)):
            cnt[0]+=1
        elif np.array_equal(np.array([0,1,0]),a_mat[:,i].reshape(num_clusters)) and np.array_equal(comp_label[j],z_mat[:,i].reshape(3)):
            cnt[1]+=1
        elif np.array_equal(np.array([0,0,1]),a_mat[:,i].reshape(num_clusters)) and np.array_equal(comp_label[j],z_mat[:,i].reshape(3)):
            cnt[2]+=1
    print(cnt/total_comp_cnt[j])

[0. 1. 0.]
[0.02285714 0.         0.97714286]
[0.70694864 0.29305136 0.        ]


# Problem 9

In [4]:
# Geometric insight into how the cluster centers m_k relate to u_mat_proj
# m_k (3xm)
# u_mat_proj (mx6)

# Distance computation between each cluster center and projected basis vector (u_mat_proj)

for cluster_idx in range(num_clusters):
    for basis_idx in range(6):
        norm_dist = np.linalg.norm(m_k[cluster_idx,:]-u_mat_proj[:,basis_idx]) / np.linalg.norm(u_mat_proj[:,basis_idx])
        angle = np.arccos(np.dot(u_mat_proj[:,basis_idx],m_k[cluster_idx,:])/(np.linalg.norm(u_mat_proj[:,basis_idx]) * np.linalg.norm(m_k[cluster_idx,:])))
        print('Cluster:',cluster_idx+1, ',Basis:', basis_idx+1, '| Normalised distance:',round(norm_dist,3), '| Angle (degrees):',round(angle*180/(np.pi),3))
    print('\n')


Cluster: 1 ,Basis: 1 | Normalised distance: 1.894 | Angle (degrees): 112.667
Cluster: 1 ,Basis: 2 | Normalised distance: 1.212 | Angle (degrees): 67.691
Cluster: 1 ,Basis: 3 | Normalised distance: 1.42 | Angle (degrees): 92.273
Cluster: 1 ,Basis: 4 | Normalised distance: 1.633 | Angle (degrees): 93.763
Cluster: 1 ,Basis: 5 | Normalised distance: 1.64 | Angle (degrees): 75.444
Cluster: 1 ,Basis: 6 | Normalised distance: 0.727 | Angle (degrees): 36.941


Cluster: 2 ,Basis: 1 | Normalised distance: 0.899 | Angle (degrees): 11.839
Cluster: 2 ,Basis: 2 | Normalised distance: 2.162 | Angle (degrees): 103.426
Cluster: 2 ,Basis: 3 | Normalised distance: 1.978 | Angle (degrees): 108.67
Cluster: 2 ,Basis: 4 | Normalised distance: 1.876 | Angle (degrees): 78.756
Cluster: 2 ,Basis: 5 | Normalised distance: 2.645 | Angle (degrees): 98.66
Cluster: 2 ,Basis: 6 | Normalised distance: 1.867 | Angle (degrees): 79.661


Cluster: 3 ,Basis: 1 | Normalised distance: 1.709 | Angle (degrees): 81.479
Cluster: 